In [11]:
import pandas as pd
import numpy as np
import datetime  
import time
import networkx as nx
import matplotlib.pyplot as plt
from itertools import chain
from matplotlib.pyplot import figure
import operator

In [89]:
convertTable = dict()
convertTable[0]= str
convertTable[1]= str
convertTable[2]= str
convertTable['ID']= str
company_id = pd.read_csv('CompanyIndex.csv', dtype= convertTable)
call_d = pd.read_csv('calls.csv', header=None, dtype= convertTable)
# emails = pd.read_csv('emails.csv', header=None, dtype= convertTable)
# meetings = pd.read_csv('meetings.csv', header=None, dtype= convertTable)
# purchases = pd.read_csv('purchases.csv', header=None, dtype= convertTable)
suspi_call = pd.read_csv('Suspicious_calls.csv', header=None, dtype= convertTable)
suspi_email = pd.read_csv('Suspicious_emails.csv', header=None, dtype= convertTable)
suspi_meetings = pd.read_csv('Suspicious_meetings.csv', header=None, dtype= convertTable)
suspi_purchases = pd.read_csv('Suspicious_purchases.csv', header=None, dtype= convertTable)

In [90]:
print('len company_id: ',len(company_id))
print('len call_d: ',len(call_d))
# print('len emails: ',len(emails))
# print('len meetings: ',len(meetings))
# print('len purchases: ',len(purchases))
print('len suspi_call: ',len(suspi_call))
# print('len suspi_email: ',len(suspi_email))
# print('len suspi_meetings: ',len(suspi_meetings))
# print('len suspi_purchases: ',len(suspi_purchases))

len company_id:  642631
len call_d:  10606835
len suspi_call:  70


In [91]:
company_id['name'] = company_id['first']+'  '+company_id['last']
company_id = company_id.drop(['first','last'], axis=1)
company_id

,ID,name
0,253,Margaret Young
1,255,Adele Burke
2,262,Zetta Unglaub
3,266,Ruthie Luzier
4,270,Christina Little
...,...,...
642626,2044645,Oscar Hutley
642627,2045457,Armand Millerbernd
642628,2047240,Duane Owens
642629,2050522,Alma Borok


เนื่องจากข้อมูลมีขนาดใหญ่มากทำให้ใช้เวลานาน เลือกเลือกมาแค่คนที่เกี่ยวข้องกับบุคคลน่าสงสัย

หา ID ของบุคคลน่าสงสัยโดยรวมรวมมาจากไฟล์ 
- Suspicious_calls
- Suspicious_emails
- Suspicious_meetings
- Suspicious_purchases

In [77]:

susCall_list = list(chain(suspi_call[0], suspi_call[2]))
susEmail_list = list(chain(suspi_email[0], suspi_email[2]))
susMeet_list =list(chain(suspi_meetings[0], suspi_meetings[2]))
susPurc_list = list(chain(suspi_purchases[0], suspi_purchases[2]))


susp_id = list(chain(susCall_list, susEmail_list, susMeet_list, susPurc_list))

susp_id = set(susp_id)
susp_id = list(susp_id)
print('จำนวนคนทั้งหมดที่อยู่ในไฟล์ผู้ต้องสงสัย ',len(susp_id), 'คน ที่มี id ดังนี้')
susp_id

จำนวนคนทั้งหมดที่อยู่ในไฟล์ผู้ต้องสงสัย  20 คน ที่มี id ดังนี้


['696047',
 '1663285',
 '505444',
 '51780',
 '28520',
 '387200',
 '728286',
 '1108217',
 '387676',
 '1886447',
 '857138',
 '1847246',
 '1642962',
 '234164',
 '969089',
 '1023940',
 '2038003',
 '1039322',
 '623027',
 '1690582']

In [78]:
# จากเดิมผู้ต้องสงสัยมี 20 คน
susp_df1 = pd.DataFrame(susp_id, columns=["id"])
susp_df1

,id
0,696047
1,1663285
2,505444
3,51780
4,28520
5,387200
6,728286
7,1108217
8,387676
9,1886447


In [93]:
susp_df1name = pd.merge(left=susp_df1[['id']],
                     right = company_id[['ID','name']], 
                     how = 'inner' ,
                     left_on = 'id',
                     right_on = 'ID'
                    )
susp_df1name = susp_df1name.drop(columns=['ID'])
susp_df1name

,id,name
0,696047,Jose Ringwald
1,1663285,Sara Ballard
2,505444,Alex Hall
3,51780,Melita Scarpaci
4,28520,Refugio Orrantia
5,387200,Augusta Sharp
6,728286,Kerstin Belveal
7,1108217,Lindsy Henion
8,387676,May Burton
9,1886447,Julie Tierno


## ผู้ต้องสงสัยที่วิเคราะห์จาก ข้อมูลการติดต่อกันผ่านไฟล์ call, email, meeting, purchase

In [101]:
su = ['1847246','2038003', '857138', '728286', '1108217', '387200', 
      '1847246', '1642962', '1023940', '234164', '696047','728286', 
      '857138' ,'1108217', '1690582' ,'1108217', '696047', '1642962', 
      '1690582', '1663285', '857138','1525352', '1600928', '1627133', 
      '1685313', '1971777', '728286', '1886447', '175354', '981554', 
      '1690582', '1886447']
su = list(set(su))
print(len(su))
su

20


['1971777',
 '696047',
 '981554',
 '1663285',
 '387200',
 '1108217',
 '728286',
 '175354',
 '1886447',
 '1600928',
 '857138',
 '1847246',
 '1642962',
 '234164',
 '1023940',
 '2038003',
 '1525352',
 '1627133',
 '1685313',
 '1690582']

In [102]:
add_su = list(set(su) - set(susp_id))
print('พบผู้น่าสงสัยเพิ่มเติมอีก ',len(add_su), 'id ดังนี้')
add_su

พบผู้น่าสงสัยเพิ่มเติมอีก  7 id ดังนี้


['1685313', '1971777', '981554', '175354', '1525352', '1600928', '1627133']

In [103]:
susp_df2 = pd.DataFrame(add_su, columns=["id_"])
susp_df2

,id_
0,1685313
1,1971777
2,981554
3,175354
4,1525352
5,1600928
6,1627133


In [104]:
susp_df2name = pd.merge(left=susp_df2[['id_']],
                     right = company_id[['ID','name']], 
                     how = 'inner' ,
                     left_on = 'id_',
                     right_on = 'ID'
                    )
susp_df2name = susp_df2name.drop(columns=['ID'])
susp_df2name 

,id_,name
0,1685313,Craig Carr
1,1971777,Loriann Gerard
2,981554,Sherrell Biebel
3,175354,Madeline Nindorf
4,1525352,Marian Ahmadi
5,1600928,Ricky Miles
6,1627133,Chang Tulip


In [105]:
all_susp = susp_id + add_su
all_susp

['696047',
 '1663285',
 '505444',
 '51780',
 '28520',
 '387200',
 '728286',
 '1108217',
 '387676',
 '1886447',
 '857138',
 '1847246',
 '1642962',
 '234164',
 '969089',
 '1023940',
 '2038003',
 '1039322',
 '623027',
 '1690582',
 '1685313',
 '1971777',
 '981554',
 '175354',
 '1525352',
 '1600928',
 '1627133']

In [106]:
all_susp = pd.DataFrame(all_susp, columns=["id"])

allsusp_df = pd.merge(left=all_susp[['id']],
                     right = company_id[['ID','name']], 
                     how = 'inner' ,
                     left_on = 'id',
                     right_on = 'ID'
                    )
allsusp_df = allsusp_df.drop(columns=['ID'])
allsusp_df


,id,name
0,696047,Jose Ringwald
1,1663285,Sara Ballard
2,505444,Alex Hall
3,51780,Melita Scarpaci
4,28520,Refugio Orrantia
5,387200,Augusta Sharp
6,728286,Kerstin Belveal
7,1108217,Lindsy Henion
8,387676,May Burton
9,1886447,Julie Tierno
